In [1]:
%load_ext autoreload
%autoreload 2

In [124]:
import numpy as np
from numba import jit
from __future__ import division
from cs231n.classifiers.softmax import softmax_loss_naive, softmax_loss_vectorized

In [87]:
from itertools import product

In [127]:
N = 100
D = 1000
C = 10
W = np.arange(C*D).reshape(D,C).astype(float)
X = np.arange(N*D).reshape(N,D).astype(float)
y = np.random.randint(0,C,size=N)
reg = 1.

In [117]:
num_train = X.shape[0]
scores = X.dot(W)  # NxC
correct_labels = (np.array(range(len(y))), y)  # locations in scores where j = y[i]
correct_class_scores = scores[correct_labels]  # Nx1

M = scores - correct_class_scores[:, np.newaxis] + 1  # NxC margins
not_thresh = np.where(M <= 0)
Lij = M
Lij[not_thresh] = 0
Lij[correct_labels] = 0

loss = np.sum(Lij)/X.shape[0] + 0.5*reg*np.sum(W * W)

TMP = np.ones(M.shape)
TMP[not_thresh] = 0
TMP[correct_labels] = 0 # NxC; 1 where margin > 0
print TMP
# contributions from j != y[i]
dW = X.T.dot(TMP)  # np.einsum('ij,ik->kj',TMP,X)

# contributions from j = y[i]
cnts = TMP.sum(axis=-1)
print cnts
print dW
for i in xrange(num_train):
    dW[:, y[i]] -= cnts[i]*X[i]
print dW
dW1 = dW/num_train + 2*reg*W

[[ 0.  1.  1.]
 [ 0.  1.  1.]
 [ 0.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]]
[ 2.  2.  0.  1.  1.]
[[   0.   10.   80.]
 [   0.   12.   84.]
 [   0.   14.   88.]
 [   0.   16.   92.]
 [   0.   18.   96.]
 [   0.   20.  100.]
 [   0.   22.  104.]
 [   0.   24.  108.]
 [   0.   26.  112.]
 [   0.   28.  116.]]
[[ -20.  -60.   80.]
 [ -24.  -60.   84.]
 [ -28.  -60.   88.]
 [ -32.  -60.   92.]
 [ -36.  -60.   96.]
 [ -40.  -60.  100.]
 [ -44.  -60.  104.]
 [ -48.  -60.  108.]
 [ -52.  -60.  112.]
 [ -56.  -60.  116.]]


In [120]:
num_train = X.shape[0]
scores = X.dot(W)  # NxC
correct_labels = (np.array(range(len(y))), y)  # locations in scores where j = y[i]
correct_class_scores = scores[correct_labels]  # Nx1

M = scores - correct_class_scores[:, np.newaxis] + 1  # NxC margins
not_thresh = np.where(M <= 0)
Lij = M
Lij[not_thresh] = 0
Lij[correct_labels] = 0

loss = np.sum(Lij)/X.shape[0] + 0.5*reg*np.sum(W * W)

TMP = np.ones(M.shape)
TMP[not_thresh] = 0
TMP[correct_labels] = 0
TMP[correct_labels] =  -1*TMP.sum(axis=-1) # NxC; 1 where margin > 0
print TMP
# contributions from j != y[i]
dW = X.T.dot(TMP)  # np.einsum('ij,ik->kj',TMP,X)
print dW
# contributions from j = y[i]
# cnts = TMP.sum(axis=-1)
# for i in xrange(num_train):
#     dW[:, y[i]] -= cnts[i]*X[i]

dW2 = dW/num_train + 2*reg*W

[[-2.  1.  1.]
 [-2.  1.  1.]
 [ 0.  0. -0.]
 [ 0. -1.  1.]
 [ 0. -1.  1.]]
[[ -20.  -60.   80.]
 [ -24.  -60.   84.]
 [ -28.  -60.   88.]
 [ -32.  -60.   92.]
 [ -36.  -60.   96.]
 [ -40.  -60.  100.]
 [ -44.  -60.  104.]
 [ -48.  -60.  108.]
 [ -52.  -60.  112.]
 [ -56.  -60.  116.]]


In [121]:
np.allclose(dW1, dW2)

True

In [98]:
dW = np.zeros_like(W)
scores = X.dot(W)
max_scores = np.max(scores, axis=1, keepdims=True)
exp_scores = np.exp(scores - max_scores)
probs = exp_scores/np.sum(exp_scores, axis=1, keepdims=True)

label_locs = tuple([np.arange(X.shape[0]), np.array([y[i] for i in xrange(X.shape[0])],dtype='int64')])
label_locs = zip(*[(i,y[i]) for i in xrange(X.shape[0])])
loss = -np.sum(np.log(probs[label_locs]))/X.shape[0] + 0.5*reg*np.sum(W*W)
probs[label_locs] = probs[label_locs]-1.
dW = X.T.dot(probs) #np.einsum('ij,ik->kj',probs,X)
dW = dW/X.shape[0] + 2*reg*W

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:9: RuntimeWarning: divide by zero encountered in log


In [72]:
probs[label_locs] = probs[label_locs]-1.

In [79]:
np.allclose(dW,dW1)

True

In [80]:
probs.shape

(10L, 3L)

In [100]:
%%timeit
loss1, dW1 = softmax_loss_naive(W,X,y,reg)

100 loops, best of 3: 4.84 ms per loop


In [123]:
%%timeit
loss2, dW2 = softmax_loss_vectorized(W,X,y,reg)

1000 loops, best of 3: 218 µs per loop


cs231n\classifiers\softmax.py:81: RuntimeWarning: divide by zero encountered in log
  loss = -np.sum(np.log(probs[label_locs]))/X.shape[0] + 0.5*reg*np.sum(W*W)


In [128]:
%%timeit
loss2, dW2 = softmax_loss_vectorized(W,X,y,reg)

100 loops, best of 3: 12.9 ms per loop


In [85]:
np.allclose(dW1,dW2)

True

In [62]:
loss1

614.00000454009603

In [9]:
scores = X[0].dot(W)
max_score = np.max(scores)
exp_scores = np.exp(scores - max_score)
probs = exp_scores/np.sum(exp_scores)

In [13]:
probs

0.99995460007033099

In [12]:
np.log(probs[y[0]])

TypeError: 'int' object has no attribute '__getitem__'

In [13]:
a[[(1,0),(1,1)]]

array([6, 1])

In [6]:
a

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

In [8]:
np.argmax(a,axis=1)

array([4, 4, 4], dtype=int64)

## loop

In [233]:
reg = 1.
X = np.arange(15).reshape(3,5)
W = np.arange(10).reshape(5,2)
y = np.array([1,0,0])
scores = X.dot(W)
correct_labels = (np.array(range(len(y))),y)
correct_scores = np.array([scores[i,k] for i,k in enumerate(y)])

In [239]:
dW = np.zeros(W.shape)  # initialize the gradient as zero

# compute the loss and the gradient
num_classes = W.shape[1]
num_train = X.shape[0]

loss = 0.0
for i in xrange(num_train):
    scores = X[i].dot(W)
    correct_class_score = scores[y[i]]
    for j in xrange(num_classes):
        cnt = 0
        if j == y[i]:
            continue
        margin = scores[j] - correct_class_score + 1  # note delta = 1
        if margin > 0:
            print margin
            loss += margin
            dW[:, j] += X[i]
            cnt -= 1
    dW[:, y[i]] += cnt*X[i]
    
print dW
loss /= num_train
dW /= num_train

# Add regularization to the loss.
loss += 0.5 * reg * np.sum(W * W)
dW += 2*reg*W

36
61
[[-15.  15.]
 [-17.  17.]
 [-19.  19.]
 [-21.  21.]
 [-23.  23.]]


In [242]:
dW[:,(0,1,0)]

array([[ -5.        ,   7.        ,  -5.        ],
       [ -1.66666667,  11.66666667,  -1.66666667],
       [  1.66666667,  16.33333333,   1.66666667],
       [  5.        ,  21.        ,   5.        ],
       [  8.33333333,  25.66666667,   8.33333333]])

In [241]:
dW2

array([[ -5.        ,   7.        ],
       [ -1.66666667,  11.66666667],
       [  1.66666667,  16.33333333],
       [  5.        ,  21.        ],
       [  8.33333333,  25.66666667]])

# vector

In [176]:
X = np.arange(15).reshape(3,5)
W = np.arange(10).reshape(5,2)
y = np.array([1,0,0])
scores = X.dot(W)
correct_labels = (np.array(range(len(y))),y)
correct_scores = np.array([scores[i,k] for i,k in enumerate(y)])

In [177]:
scores

array([[ 60,  70],
       [160, 195],
       [260, 320]])

In [178]:
M

array([[ 0,  0],
       [ 0, 36],
       [ 0, 61]])

In [179]:
M = scores - correct_scores[:,np.newaxis] + 1
not_thresh = np.where(M <= 0)

In [193]:
L = M.copy()

L[not_thresh] = 0
L[correct_labels] = 0
print L
loss = np.sum(L)/X.shape[0] + 0.5*reg*np.sum(W * W)

[[ 0  0]
 [ 0 36]
 [ 0 61]]


In [185]:
np.sum(L)/3.

32.333333333333336

In [195]:
dW = np.zeros(W.shape)
TMP = np.ones(M.shape)
TMP[not_thresh] = 0
TMP[correct_labels] = 0

In [211]:
%%timeit
(X.T).dot(TMP)

The slowest run took 75.02 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 1.19 µs per loop


In [212]:
%%timeit
np.einsum('ij,ik->kj',TMP,X)

The slowest run took 10.72 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 2.05 µs per loop


In [237]:
TMP[not_thresh] = 0
TMP[correct_labels] = 0
dW = (X.T).dot(TMP) # np.einsum('ij,ik->kj',TMP,X)
num_train = X.shape[0]
cnts = TMP.sum(axis=-1)
for i in xrange(num_train):
    dW[:, y[i]] -= cnts[i]*X[i]
# dWyi = -1*(X.T).dot(TMP[np.newaxis,:])

dW = dW/num_train + 2*reg*W

In [243]:
X

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

In [249]:
np.einsum('i,ij->ij',cnts,X)

array([[  0.,   0.,   0.,   0.,   0.],
       [  5.,   6.,   7.,   8.,   9.],
       [ 10.,  11.,  12.,  13.,  14.]])

In [250]:
cnts

array([ 0.,  1.,  1.])

In [204]:
np.einsum('ij,ik->ikj',TMP,X)

array([[[  0.,   0.],
        [  0.,   0.],
        [  0.,   0.],
        [  0.,   0.],
        [  0.,   0.]],

       [[  0.,   5.],
        [  0.,   6.],
        [  0.,   7.],
        [  0.,   8.],
        [  0.,   9.]],

       [[  0.,  10.],
        [  0.,  11.],
        [  0.,  12.],
        [  0.,  13.],
        [  0.,  14.]]])

In [205]:
np.einsum('ij,ik->kj',TMP,X)

array([[  0.,  15.],
       [  0.,  17.],
       [  0.,  19.],
       [  0.,  21.],
       [  0.,  23.]])

In [122]:
M[loc]

array([-9])

In [82]:
scores.diagonal() = 0

SyntaxError: can't assign to function call (<ipython-input-82-2e82b29b698d>, line 1)

In [78]:
scores[:,np.array([0,0])]

array([[ 60,  60],
       [160, 160],
       [260, 260]])

In [62]:
scores[0,0]

60

In [55]:
scores[np.array([(0),(0)])]

array([[60, 70],
       [60, 70]])

In [72]:
correct_scores = np.array([scores[i,k] for i,k in enumerate(y)])

In [73]:
correct_scores

array([ 70, 160, 260])

In [22]:
np.__version__

'1.10.2'

In [9]:
print(X)
print(W)

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]]
[[0 1]
 [2 3]
 [4 5]
 [6 7]
 [8 9]]


In [23]:

X.dot(W)

array([[ 60,  70],
       [160, 195],
       [260, 320]])

In [25]:

X[1].dot(W)

array([160, 195])

In [19]:
%%timeit
np.einsum('ij,jk', X,W)

The slowest run took 13.23 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 1.44 µs per loop


In [12]:
scores

array([160, 195])

In [235]:
@jit
def jit_loop(train, test):
    """
    Compute the distance between each test point in test and each training point
    in train using a nested loop over both the training data and the
    test data.

    Inputs:
    - test: A numpy array of shape (num_test, D) containing test data.

    Returns:
    - dists: A numpy array of shape (num_test, num_train) where dists[i, j]
      is the Euclidean distance between the ith test point and the jth training
      point. """
    num_test = test.shape[0]
    num_train = train.shape[0]
    dists = np.zeros((num_test, num_train))
    dim = test.shape[1]
    for i in range(num_test):
        for j in range(num_train):
            sum_ = 0
            for k in range(dim):
                x = train[j, k] - test[i, k]
                sum_ += x*x
            dists[i,j] = sum_
    return np.sqrt(dists)

def compute_distances_two_loops(data, X):
    """
    Compute the distance between each test point in X and each training point
    in data using a nested loop over both the training data and the
    test data.

    Inputs:
    - X: A numpy array of shape (num_test, D) containing test data.

    Returns:
    - dists: A numpy array of shape (num_test, num_train) where dists[i, j]
      is the Euclidean distance between the ith test point and the jth training
      point. """
    num_test = X.shape[0]
    num_train = data.shape[0]
    dists = np.zeros((num_test, num_train))
    for i in xrange(num_test):
        for j in xrange(num_train):
            dists[i,j] = np.sum((data[j] - X[i])**2.)
    return np.sqrt(dists)

def compute_distances_one_loop(data, X):
    """
    Compute the distance between each test point in X and each training point
    in data using a single loop over the test data.

    Input / Output: Same as compute_distances_two_loops """
    num_test = X.shape[0]
    num_train = data.shape[0]
    dists = np.zeros((num_test, num_train))
    d2 = np.sum(data**2, axis=1)
    for i in xrange(num_test):
        dists[i,:] = d2 + np.sum(X[i,:]**2.) - 2.*np.einsum('ij,j->i', data, X[i,:])
    return np.sqrt(dists)

def compute_distances_no_loops_bad(data, X):
    """
    Compute the distance between each test point in X and each training point
    in data using no explicit loops.

    Input / Output: Same as compute_distances_two_loops"""
    return np.sqrt(np.sum((data - X[:, np.newaxis])**2.,axis=-1))

def compute_distances_no_loops(data, X):
    """
    Compute the distance between each test point in X and each training point
    in data using no explicit loops.

    Input / Output: Same as compute_distances_two_loops"""
    return np.sqrt(np.sum(X**2, axis=1, keepdims=True) + np.sum(data**2, axis=1) - 2.*np.einsum('ij,kj->ik',X,data))

In [245]:
np.sqrt(np.sum(X**2, axis=1, keepdims=True) + np.sum(data**2, axis=1) - 2.*np.einsum('ij,kj->ik',X,data))

array([[ 6280.25469229,  6412.55292376,  6383.29068741, ...,
         6487.92170113,  6341.13152363,  6175.24274827],
       [ 6377.76426344,  6260.5168317 ,  6459.86640729, ...,
         6307.28261615,  6355.12493976,  6285.49290032],
       [ 6285.34891633,  6388.13564352,  6360.75184235, ...,
         6282.60216789,  6369.49534893,  6234.88371985],
       ..., 
       [ 6351.24523224,  6457.6684647 ,  6387.41285655, ...,
         6302.57621929,  6441.8261386 ,  6360.20219804],
       [ 6307.5241577 ,  6290.77666111,  6427.11117066, ...,
         6349.16388196,  6308.64827043,  6315.5979131 ],
       [ 6420.03816188,  6309.68319331,  6311.81772551, ...,
         6404.38888576,  6342.34310015,  6407.09840724]])

In [246]:
np.sqrt(np.sum(X**2, axis=1)[:,np.newaxis] + np.sum(data**2, axis=1) - 2.*np.einsum('ij,kj->ik',X,data))

array([[ 6280.25469229,  6412.55292376,  6383.29068741, ...,
         6487.92170113,  6341.13152363,  6175.24274827],
       [ 6377.76426344,  6260.5168317 ,  6459.86640729, ...,
         6307.28261615,  6355.12493976,  6285.49290032],
       [ 6285.34891633,  6388.13564352,  6360.75184235, ...,
         6282.60216789,  6369.49534893,  6234.88371985],
       ..., 
       [ 6351.24523224,  6457.6684647 ,  6387.41285655, ...,
         6302.57621929,  6441.8261386 ,  6360.20219804],
       [ 6307.5241577 ,  6290.77666111,  6427.11117066, ...,
         6349.16388196,  6308.64827043,  6315.5979131 ],
       [ 6420.03816188,  6309.68319331,  6311.81772551, ...,
         6404.38888576,  6342.34310015,  6407.09840724]])

In [72]:
a = np.zeros((i,i/2))
np.subtract(data, X, a)

ValueError: operands could not be broadcast together with shapes (6,5) (3,5) (6,3) 

In [157]:
2.5+7.4

9.9

In [159]:
a=0

In [160]:
a = data - X[:, np.newaxis]

In [153]:
a.shape

(500L, 5000L, 370L)

In [97]:
from math import log

In [164]:
'{:e}'.format(500*5000*3700*64//8)

'7.400000e+10'

In [94]:
a.shape

(50L, 500L, 370L)

In [240]:
i = 5000
j = 3700
data = np.random.randint(0,256,(i,j)).astype('int')
X = np.random.randint(0,256,(i/10,j)).astype('int')

In [241]:
%%timeit
jit_loop(data,X)

1 loops, best of 3: 7.29 s per loop


In [242]:
%%timeit
compute_distances_no_loops(data,X)

1 loops, best of 3: 3.1 s per loop


In [243]:
%%timeit
compute_distances_one_loop(data,X)

1 loops, best of 3: 3.07 s per loop


In [244]:
%%timeit
compute_distances_two_loops(data,X)

1 loops, best of 3: 25.4 s per loop


In [188]:
x2 = np.sum(data**2, axis=1, keepdims=True)
y2 = np.sum(X**2, axis=1)
xy = np.dot(data, X.T)
dist = np.sqrt(x2 - 2*xy + y2)

In [170]:
print data.shape
print X.shape

(5000L, 370L)
(500L, 370L)


In [171]:
xy.shape

(5000L, 500L)

In [220]:
dist = compute_distances_no_loops(data,X)

In [221]:
dist2 = jit_loop(data,X)

In [222]:
np.allclose(dist, dist2)

True